## Evaluate GS accuracy using `MM4LMM`

In [ ]:
#set up environment
library(tidyverse)
library(lme4)
library(MM4LMM)
library(corpcor)

#for a trait:
#  calculate blues 
#  extract kinship to match blues and make positive definite
#  fit mixed model for add + dom
#  fit add and dom separately
#  calculate lr ratio for adding full - each reduced model
#  use full model to evaluate accuracy (5-fold, 20 times)

calculateBlues <- function(trait) {
  model = as.formula(paste(trait,"~ germplasmName + (1|studyName/replicate)"))
  modelFit <- lmer(model, data = pheno)
  modelSummary <- summary(modelFit)
  fixedEffects <- fixef(modelFit)
  intercept <- modelSummary$coefficients[1]
  bluevals <- fixedEffects + c(0, rep(fixedEffects[1], length(fixedEffects) - 1))
  taxonName <- levels(modelFit@frame$germplasmName)
  names(bluevals) <- taxonName
  bluevals
}

calculateBlups <- function(trait) {
  model = as.formula(paste(trait,"~ (1|germplasmName) + (1|studyName/replicate)"))
  modelFit <- lmer(model, data = pheno)
  modelSummary <- summary(modelFit)
  intercept <- modelSummary$coefficients[1]
  blupvals <- intercept + ranef(modelFit)$germplasmName$`(Intercept)` 
  taxonName <- levels(modelFit@frame$germplasmName)
  names(blupvals) <- taxonName
  blupvals
}

#output: fullModel has both additive and dominance GRMs
#   The addModel and domModel have only additive and dominance GRMs, respectively
#   lrAdd is the likelihood ration test comparing the full model to the dominance model to test additive after dominance
#   lrDom tests the effect of adding dominance to an additive model
#   blues, addvar, and domvar are there to facilitate passing them to the performGp method
varianceComponents <- function(bluevals, kinAdd, kinDom) {
  ndx.add <- match(names(bluevals), kinAdd$taxon)
  kinAdd2 <- as.matrix(kinAdd[ndx.add,ndx.add + 1])
  ndx.dom <- match(names(bluevals), kinDom$taxon)
  kinDom2 <- as.matrix(kinDom[ndx.dom,ndx.dom + 1])
  #addvar <- corpcor::make.positive.definite(kinAdd2)
  #domvar <- corpcor::make.positive.definite(kinDom2)
  addvar <- kinAdd2
  domvar <- kinDom2
  rownames(addvar) <- colnames(addvar)
  rownames(domvar) <- colnames(domvar)
  errvar = diag(57)
  mmresult <- MMEst(Y = bluevals, VarList = list(add = addvar, dom = domvar,error = errvar))
  mmadd <- MMEst(Y = bluevals, VarList = list(add = addvar, error = errvar))
  mmdom <- MMEst(Y = bluevals, VarList = list(dom = domvar,error = errvar))
  lrAdd <- -2*(mmdom$NullModel$`LogLik (Reml)` - mmresult$NullModel$`LogLik (Reml)`)
  lrDom <- -2*(mmadd$NullModel$`LogLik (Reml)` - mmresult$NullModel$`LogLik (Reml)`)
  list(fullModel = mmresult, addModel = mmadd, domModel = mmdom, lrAdd = lrAdd, lrDom = lrDom, blues = bluevals, addvar = addvar, domvar = domvar)
}

#performs 10-fold genomic prediction cross validation 50 times
#output is the correlation of the original and predicted test values
performGp <- function(varcompResult) {
  allvals <- varcompResult$blues
  corrvals <- vector("numeric", 500)
  #randomly choose 11 samples to be the test set and train on the remaining 46
  ntaxa = length(allvals)
  testSize = floor(ntaxa/10)
  testIndex <- seq(1, ntaxa, testSize)
  errvar = diag(ntaxa)
  for (rep in 1:50) {
    randomIndex <- sample(1:ntaxa)
    for (fold in 1:10) {
      missIndex <- randomIndex[testIndex[fold]:(testIndex[fold+1] - 1)]
      testvals <- allvals[missIndex]
      trainvals <- allvals[-missIndex]
      #remove rows from the Z matrix corresponding to the test taxa
      zmatrix <- diag(ntaxa)[-missIndex,]
      ZL <- list(zmatrix,zmatrix,zmatrix)
      mmresult <- MMEst(Y = trainvals, ZList = ZL, VarList = list(add = varcompResult$addvar, dom = varcompResult$domvar,error = errvar))
      kinblups <- MMBlup(Y = trainvals, ZList = ZL, VarList = list(add = varcompResult$addvar, dom = varcompResult$domvar,error = errvar), ResMM = mmresult)
      corrvals[fold + (rep - 1) * 5] = cor(kinblups$add[missIndex] + kinblups$dom[missIndex], testvals)
    }
  }
  c(mean(corrvals), sd(corrvals))
}

#Analyze cassava data
#import data
pheno <- read_csv(file = "/Users/pjbra/temp/cassava-phg/HMII_IITA_57_phenotypes.csv")

#for all traits
#load kinship matrices
#allChrAdd <- read_delim(file = "/Users/pjbra/temp/cassava-phg/allchr.add.kinship.txt", delim = "\t", skip = 3, col_names = c("taxon"))
allChrAdd <- read_delim(file = "C:\\Users\\pjbra\\temp\\cassava-phg\\allchrRamuAddKinship.txt", delim = "\t", skip = 3, col_names = c("taxon"))
allChrDom <- read_delim(file = "/Users/pjbra/temp/cassava-phg/allchr.dom.kinship.txt", delim = "\t", skip = 3, col_names = c("taxon"))

phgAdd <- read_delim(file = "C:\\Users\\pjbra\\temp\\cassavaPhgAddKinship.txt", delim = "\t", skip = 3, col_names = c("taxon"))
phgDom <- read_delim(file = "C:\\Users\\pjbra\\temp\\cassavaPhgDomKinship.txt", delim = "\t", skip = 3, col_names = c("taxon"))

# test logFYLD, logDYLD, RTNO, DM 
blues <- calculateBlues("DM")
varcomp <- varianceComponents(blues, phgAdd, phgDom)
performGp(varcomp)

varcomp <- varianceComponents(blues, allChrAdd, allChrDom)
performGp(varcomp)

blups <- calculateBlups("DM")
varcomp <- varianceComponents(blups, phgAdd, phgDom)
performGp(varcomp)

varcomp <- varianceComponents(blups, allChrAdd, allChrDom)
performGp(varcomp)

#test random imputation
phgAddRandom <- read_delim(file = "/Users/pjbra/temp/cassava-phg/cassavaPhgAddKinshipRandomImputation.txt", delim = "\t", skip = 3, col_names = c("taxon"))
ndx.add <- match(names(blues), phgAddRandom$taxon)
kinAddRandom <- as.matrix(phgAddRandom[ndx.add,ndx.add + 1])
rownames(kinAddRandom) <- colnames(kinAddRandom)
errvar = diag(57)
mmresult <- MMEst(Y = blues, VarList = list(add = kinAddRandom, error = errvar))
names(varcomp)
varcomp$addvar[1:7,1:7]
kinAddRandom[1:7,1:7]

ramuKin <- read_delim(file = "/Users/pjbra/temp/cassava-phg/allchrRamuAddKinshipRandom.txt", delim = "\t", skip = 3, col_names = c("taxon"))
ramuDom <- read_delim(file = "C:\\Users\\pjbra\\temp\\cassava-phg\\allchr.dom.kinship.txt", delim = "\t", skip = 3, col_names = c("taxon"))
ndx.add <- match(names(blues), ramuKin$taxon)
kinAddRandom <- as.matrix(ramuKin[ndx.add,ndx.add + 1])
rownames(kinAddRandom) <- colnames(kinAddRandom)
errvar = diag(57)
mmresult <- MMEst(Y = blues, VarList = list(add = kinAddRandom, error = errvar))
names(varcomp)
varcomp$addvar[1:7,1:7]
kinAddRandom[1:7,1:7]

result <- varianceComponents(blues, ramuKin, ramuDom)
result$fullModel
result$addModel
result$domModel
result$lrAdd
result$lrDom

hapAddKin <- read_delim(file = "/Users/pjbra/temp/cassava-phg/cassavaHapPhgAddKinship.txt", delim = "\t", skip = 3, col_names = c("taxon"))
hapDomKin <- read_delim(file = "/Users/pjbra/temp/cassava-phg/cassavaHapPhgDomKinship.txt", delim = "\t", skip = 3, col_names = c("taxon"))

blues <- calculateBlues("DM")
varcomp <- varianceComponents(blues, hapAddKin, hapDomKin)
performGp(varcomp)

#test random imputation
varianceComponents_add <- function(bluevals, kinAdd) {
  ndx.add <- match(names(bluevals), kinAdd$taxon)
  kinAdd2 <- as.matrix(kinAdd[ndx.add,ndx.add + 1])
  #addvar <- corpcor::make.positive.definite(kinAdd2)
  addvar <- kinAdd2
  rownames(addvar) <- colnames(addvar)
  errvar = diag(57)
  MMEst(Y = bluevals, VarList = list(add = addvar, error = errvar))
}

blues <- calculateBlues("DM")
varianceComponents_add(blues, allChrDom)
varcomp <- varianceComponents(blues, phgAdd, phgDom)
varcomp$fullModel

testmatrix = as.matrix(ramuKin[,-1])
rownames(testmatrix) <- colnames(testmatrix)
isSymmetric(testmatrix)
corpcor::is.positive.definite(testmatrix)

